In [1]:
import pandas as pd
# import polars as pl

In [2]:
data_path = r"C:\Users\parth\2023\Fall 2023\Projects\docker_hack\data\data.csv"
# data_path = "../../../data/csv_dumps/data.csv"

In [3]:
df = pd.read_csv(data_path)
# df = pl.read_csv(data_path)#, encoding='latin-1')
df.head()

,first,actors,details,boxOffice,techspecs
0,"[{'title': 'Mobile Two'}, {'genres': ['Drama']...",[{'image': 'https://m.media-amazon.com/images/...,[{'title-details-releasedate': {'val': ['Septe...,[],[{'title-techspec_runtime': {'val': '1 hour 30...
1,"[{'title': 'Giving Becky a Chance'}, {'poster_...",[{'image': 'https://m.media-amazon.com/images/...,[{'title-details-releasedate': {'val': ['June ...,[],[{'title-techspec_runtime': {'val': '50 minute...
2,"[{'title': 'Promises'}, {'poster_link': 'https...",[],[{'title-details-releasedate': {'val': ['2022 ...,"[{'title-boxoffice-budget': {'val': '$8,000 (e...",[{'title-techspec_runtime': {'val': '14 minute...
3,"[{'title': 'Sin'}, {'poster_link': 'https://m....",[{'image': 'https://m.media-amazon.com/images/...,[{'title-details-releasedate': {'val': ['Octob...,[],[{'title-techspec_runtime': {'val': '50 minute...
4,"[{'title': 'The Miseducation of Bindu'}, {'pos...",[{'image': 'https://m.media-amazon.com/images/...,[{'title-details-releasedate': {'val': ['Octob...,[],[{'title-techspec_runtime': {'val': '1 hour 32...


In [26]:
df["details"][0]

"[{'title-details-releasedate': {'val': ['September 2, 1975 (United States)'], 'link': ['/title/tt0073397/releaseinfo?ref_=tt_dt_rdat']}}, {'title-details-origin': {'val': ['United States'], 'link': ['/search/title/?country_of_origin=US&ref_=tt_dt_cn']}}, {'title-details-languages': {'val': ['English'], 'link': ['/search/title?title_type=feature&primary_language=en&sort=moviemeter,asc&ref_=tt_dt_ln']}}, {'title-details-companies': {'val': ['Mark VII Ltd.', 'Universal Television'], 'link': ['/company/co0137316?ref_=tt_dt_co_1', '/company/co0046592?ref_=tt_dt_co_2']}}]"

In [4]:
#for debug
# eval(df["boxOffice"][17])
eval(df["first"][17])

[{'title': 'Listening in the Dark'},
 {'poster_link': 'https://m.media-amazon.com/images/M/MV5BY2U2MzhjNTctMWFmMS00YzE5LWIyMDMtZmEwZDMwNDU4OWZiXkEyXkFqcGdeQXVyNDA4MDE4Nw@@._V1_QL75_UY281_CR151,0,190,281_.jpg'},
 {'genres': ['Documentary', 'Short']},
 {'plot': 'Human beings are horribly blind to the effect that they have on the natural environment. Yet all around us those repercussions reverberate.'}]

In [5]:
def make_new_columns_boxOffice(curr_df,col_name, list_of_keys):
    """
    params will be(col_name,[list_of_keys]) ,
    the names in the list should be same as keys
    """
    if(col_name not in curr_df.columns):
        print("No such column exists in the df")
    for key in list_of_keys:
        curr_df[key] = ""
    for i in range(len(curr_df)):
        response_list = eval(curr_df["boxOffice"][i])
        if(response_list):
            response = response_list
            for it in response:
                for key in list_of_keys:
                    if key in it:
                        curr_df[key][i] = it[key]["val"]
                        

    return curr_df
    
    # delete the original column 

def make_new_columns_first(curr_df, col_name, list_of_keys):
    """
    params will be(col_name,[list_of_keys]) ,
    the names in the list should be same as keys
    """
    if (col_name not in curr_df.columns):
        print("No such column exists in the df")
    for key in list_of_keys:
        curr_df[key] = ""
    for i in range(len(curr_df)):
        response_list = eval(curr_df["first"][i])
        if (response_list):
            response = response_list
            for it in response:
                for key in list_of_keys:
                    if key in it:
                        curr_df[key][i] = it[key]
                        

    return curr_df
    
    # delete the original column 

# def insert_column_num_actors(curr_df):
#     """
#     Inserts a new column that states how many actors are in a particular movie.
#     """
#     curr_df['num_actors'] = pd.Series(dtype='int')
#     for i in range(len(curr_df)):
#         response_list = eval(curr_df["actors"][i])
#         if (response_list):
#             curr_df['num_actors'][i] = len(response_list)
#         #     response = response_list
#         #     for it in response:
#         #         for key in list_of_keys:
#         #             if key in it:
#         #                 curr_df[key][i] = it[key]
                        

#     return curr_df
    
    # delete the original column 
    

In [6]:
new_df = df
new_df = make_new_columns_boxOffice(new_df,"boxOffice", ["title-boxoffice-budget","title-boxoffice-cumulativeworldwidegross","title-boxoffice-grossdomestic","title-boxoffice-openingweekenddomestic"])
new_df = make_new_columns_first(new_df,'first',['title','poster_link','genres','plot','imdb-rating'])
new_df.drop(['first','boxOffice','poster_link'], axis = 1, inplace = True)

In [7]:
# pre-processing steps

# capture only the imdb rating, remove /10 part and convert to float, insert NaN where no value exists
new_df['imdb_rating'] = new_df['imdb-rating'].str.split('/').str[0]
new_df['imdb_rating'] = pd.to_numeric(new_df['imdb_rating'], errors='coerce').astype(float)

# create new column #genres, fills in 0 where no value is present
new_df['num_genres'] = new_df['genres'].apply(lambda x: len(x))

# create new column #actors, fills in 0 where no value is present
new_df['num_actors'] = new_df['actors'].apply(lambda x: len(eval(x)))

In [8]:
def money_str_to_currency(input_str):
    '''Takes an input as a str. Saves currency signs if any\n",
    saves value'''
    if(len(input_str)==0):
        return None
    #strip the str of any spaces\n",
    stripped_input_str = input_str.strip()
    split_str = stripped_input_str.split()
    money_str = split_str[0]
    currency = money_str[:1]
    # money_with_comma = money_str[1:]
    # money = money_with_comma.replace(",","")
    return currency

In [9]:
def money_str_to_int(input_str):
    '''Takes an input as a str. Saves currency signs if any\n",
    saves value'''
    if(len(input_str)==0):
        return None
    #strip the str of any spaces\n",
    stripped_input_str = input_str.strip()
    split_str = stripped_input_str.split()
    money_str = split_str[0]
    currency = money_str[:1]
    money_with_comma = money_str[1:]
    money = money_with_comma.replace(",","")
    return money

In [10]:
new_df["currency"] = None
new_df["budget_money"] = None

In [11]:
new_df["currency"] = new_df["title-boxoffice-budget"].apply(money_str_to_currency)
new_df["budget_money"] = new_df["title-boxoffice-budget"].apply(money_str_to_int)

In [12]:
df["techspecs"][0]

"[{'title-techspec_runtime': {'val': '1 hour 30 minutes'}}, {'title-techspec_soundmix': {'val': ['Mono']}}, {'title-techspec_aspectratio': {'val': ['1.33 : 1']}}]"

In [13]:
new_df["runtime"] = None
new_df["soundmix"] = None
new_df["aspect_ratio"] = None

In [14]:
def extract_techspecs(input_str):
    try:
        if(len(input_str)==0):
            return [None,None,None]
        input_list = eval(input_str)
        '''iterate on every object,
        find every key in every object,
        if you find it place the value 
        else place None'''
        runtime = None
        soundmix = None
        aspect_ratio = None
        for obj in input_list:
            if("title-techspec_runtime" in obj):
                runtime = obj["title-techspec_runtime"]["val"]
            if("title-techspec_soundmix" in obj):
                soundmix = obj["title-techspec_soundmix"]["val"]
            if("title-techspec_aspectratio" in obj):
                aspect_ratio = obj["title-techspec_aspectratio"]["val"]
    except Exception as e:
        print(input_str,e)
        sys.exit(1)
    return [runtime,soundmix,aspect_ratio]
        
            

In [15]:
for i in range(len(new_df)):
    input_str = new_df["techspecs"][i]
    runtime,soundmix,aspect_ratio = extract_techspecs(input_str)
    new_df["runtime"][i] = runtime
    new_df["soundmix"][i] = soundmix
    new_df["aspect_ratio"][i] = aspect_ratio
    

C:\Users\parth\AppData\Local\Temp\ipykernel_34444\734792789.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["runtime"][i] = runtime
C:\Users\parth\AppData\Local\Temp\ipykernel_34444\734792789.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["soundmix"][i] = soundmix
C:\Users\parth\AppData\Local\Temp\ipykernel_34444\734792789.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["aspect_ratio"][i] = aspect_rati

In [16]:
# new_df["runtime"] = new_df["techspecs"].apply(extract_techspecs)[0]
# new_df["soundmix"] = new_df["techspecs"].apply(extract_techspecs)[1]
# new_df["aspect_ratio"] = new_df["techspecs"].apply(extract_techspecs)[2]

In [17]:
new_df.columns

Index(['actors', 'details', 'techspecs', 'title-boxoffice-budget',
       'title-boxoffice-cumulativeworldwidegross',
       'title-boxoffice-grossdomestic',
       'title-boxoffice-openingweekenddomestic', 'title', 'genres', 'plot',
       'imdb-rating', 'imdb_rating', 'num_genres', 'num_actors', 'currency',
       'money', 'runtime', 'soundmix', 'aspect_ratio'],
      dtype='object')

In [18]:
def extract_columns_from_dataframe(df,column_list):
    """provide a dataframe and a column list,
    return the subset of the datafram if the column exists"""
    column_confirmed_list = []
    for column in column_list:
        if column in df.columns:
            column_confirmed_list.append(column)
        else:
            print("{} column not in the dataframe\n".format(column))
    df = df[column_confirmed_list]
    return df

In [19]:
new_df[["title"]]

,title
0,Mobile Two
1,Giving Becky a Chance
2,Promises
3,Sin
4,The Miseducation of Bindu
...,...
100394,Ron Hopper's Misfortune
100395,Coins for Christmas
100396,Alpha Delta Zatan
100397,Quadratic Equations


In [20]:
column_list = ['title', 'genres', 'plot',
       'imdb-rating', 'imdb_rating', 'num_genres', 'num_actors', 'currency',
       'money', 'runtime', 'soundmix', 'aspect_ratio']

In [21]:
extract_df = extract_columns_from_dataframe(new_df,column_list)

In [25]:
extract_df.to_csv(r"C:\Users\parth\2023\Fall 2023\Projects\movie_analysis\data\csvs\file.csv")

In [ ]:
# df["title-boxoffice-budget"][1]

In [ ]:
df.eq('').sum() 